In this notebook, we illustrate how to fit GMM with different optimization routines. A similar approach can be followed for fitting other models with these optimizers.

In [1]:
from Mixture_Models import *

#### Simulating some data

In [2]:
data = make_pinwheel(
    radial_std=0.3,
    tangential_std=0.05,
    num_classes=3,
    num_per_class=100,
    rate=0.4,
    rs=npr.RandomState(0),
)

#### Initializing the model

In [3]:
test_GMM = GMM(data)

#### Initializing the input parameters

In [4]:
npr.seed(10)
init_params = test_GMM.init_params(num_components=3, scale=0.5)
print(init_params)

{'log proportions': array([ 0.66579325,  0.35763949, -0.77270015]), 'means': array([[-0.00419192,  0.31066799],
       [-0.36004278,  0.13275579],
       [ 0.05427426,  0.00214572]]), 'sqrt_covs': array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]])}


First let's see what these unpacked true parameters look like. Since the mixture proportions and covariances are reparametrized
as described in the Readme, we show how to recover the actual parameters from their reparametrized versions.

In [5]:
print(test_GMM.unpack_params(init_params))

(array([-0.67909253, -0.98724629, -2.11758593]), array([[-0.00419192,  0.31066799],
       [-0.36004278,  0.13275579],
       [ 0.05427426,  0.00214572]]), array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]]))


###### Recovering the mixture proportions

In [6]:
# To recover the actual mixture proportions
print(np.exp(test_GMM.log_normalize(init_params["log proportions"])))
# Alternately
print(np.exp(test_GMM.unpack_params(init_params)[0]))

[0.50707694 0.37260131 0.12032174]
[0.50707694 0.37260131 0.12032174]


In [7]:
# Input a custom mixture proportion say [0.1,0.2,0.7]
init_params["log proportions"] = np.log(
    [0.1, 0.2, 0.7]
)  # i.e. we input the log proportions
print(
    np.exp(test_GMM.log_normalize(init_params["log proportions"]))
)  ## Test if we recover our input proportions.

[0.1 0.2 0.7]


###### Recovering the covariance matrices

In [8]:
# To recover the covariance matrices from their square roots
cov = []
for log_proportion, mean, cov_sqrt in zip(*test_GMM.unpack_params(init_params)):
    cov.append(cov_sqrt.T @ cov_sqrt)
print(cov)

[array([[1., 0.],
       [0., 1.]]), array([[1., 0.],
       [0., 1.]]), array([[1., 0.],
       [0., 1.]])]


In [9]:
# Say we want to the first component's initialized covariance as [[4,-0.5],[-0.5,1]]
from scipy.linalg import sqrtm

init_params["sqrt_covs"][0] = sqrtm(
    [[4, -0.75], [-0.75, 1]]
)  ## Input our custom covariance using the reparametrization A^T*A = Covariance
# Crosscheck if the covariance is changing
cov = []
for log_proportion, mean, cov_sqrt in zip(*test_GMM.unpack_params(init_params)):
    cov.append(cov_sqrt.T @ cov_sqrt)
print(cov)

[array([[ 4.  , -0.75],
       [-0.75,  1.  ]]), array([[1., 0.],
       [0., 1.]]), array([[1., 0.],
       [0., 1.]])]


#### Fitting the model with the above initialization parameters

Gradient Descent with momentum

In [10]:
params_store = test_GMM.fit(
    init_params,
    opt_routine="grad_descent",
    learning_rate=0.0005,
    mass=0.9,
    maxiter=100,
    tol=1e-7,
)

Log likelihood -722.4161069308387
Log likelihood -721.1705366046117
Log likelihood -718.8246652036032
Log likelihood -715.5301044696276
Log likelihood -711.447179403069
Log likelihood -706.7491006708486
Log likelihood -701.6268869728219
Log likelihood -696.2938895297725
Log likelihood -690.9882711143798
Log likelihood -685.9709407902037
Log likelihood -681.5154284034992
Log likelihood -677.8855644750276
Log likelihood -675.2978406413845
Log likelihood -673.8698345270402
Log likelihood -673.5654534230541
Log likelihood -674.1598789846468
Log likelihood -675.2534205513352
Log likelihood -676.3518710435866
Log likelihood -676.9982189288902
Log likelihood -676.904272079762
Log likelihood -676.0209669724117
Log likelihood -674.5169305331922
Log likelihood -672.6854863454756
Log likelihood -670.8326365146306
Log likelihood -669.1938018253461
Log likelihood -667.8987744414289
Log likelihood -666.9776100087684
Log likelihood -666.3885341891558
Log likelihood -666.0505839797341
Log likelihood -